In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets einops wandb
!pip install -q bitsandbytes

In [1]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 42.8 MB/s eta 0:00:00


In [2]:
pip install -U bitsandbytes>=0.46.1

In [3]:
# Import necessary libraries
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset

In [4]:
logging.set_verbosity_info()
logger = logging.get_logger("transformers")

In [5]:
# Check GPU availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

PyTorch version: 2.9.0+cu128
CUDA available: True
CUDA device: NVIDIA A100-SXM4-40GB
GPU Memory: 42.41 GB


In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
dataset_name = "timdettmers/openassistant-guanaco"
dataset = load_dataset(dataset_name, split="train")

print(f"Guanaco Dataset loaded successfully!")
print(f"\nDataset size: {len(dataset):,} examples")
print(f"\nDataset features:")
print(dataset.features)

README.md:   0%|          | 0.00/395 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


openassistant_best_replies_train.jsonl:   0%|          | 0.00/20.9M [00:00<?, ?B/s]

openassistant_best_replies_eval.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/9846 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/518 [00:00<?, ? examples/s]

Guanaco Dataset loaded successfully!

Dataset size: 9,846 examples

Dataset features:
{'text': Value('string')}


In [10]:
print("------------Sample1-------------")
print()
i = 0
print(f"Index: {i}")
print(f"Text: {dataset[i]["text"]}")

------------Sample1-------------

Index: 0
Text: ### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.

Recent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limite

In [11]:
load_in_4bit = True
bnb_4bit_compute_dtype = torch.float16
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

In [12]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_use_double_quant=use_nested_quant
)

In [13]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

print('---------------------------------------')
print(f"Loading {model_name}, Please Wait ...")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)

model.config.use_cache = False
model.gradient_checkpointing_enable()

print("\n Model loaded successfully!")
print(f"Model: {model_name}")

---------------------------------------
Loading meta-llama/Meta-Llama-3-8B-Instruct, Please Wait ...


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/8afb486c1db24fe5011ec46dfbe5b5dccdb575c2/config.json
`torch_dtype` is deprecated! Use `dtype` instead!
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "dtype": "float16",
  "eos_token_id": 128009,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": null,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_parameters": {
    "rope_theta": 500000.0,
    "rope_type": "default"
  },
  "tie_word_embeddings": false,
  "transformers_version": "5.0.0",
  "use_cache": true,
  "vocab_size": 128256

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/8afb486c1db24fe5011ec46dfbe5b5dccdb575c2/model.safetensors.index.json


...

Generate config GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "output_attentions": false,
  "output_hidden_states": false,
  "use_cache": true
}

We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/8afb486c1db24fe5011ec46dfbe5b5dccdb575c2/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128009
  ],
  "max_length": 4096,
  "temperature": 0.6,
  "top_p": 0.9
}




 Model loaded successfully!
Model: meta-llama/Meta-Llama-3-8B-Instruct


In [14]:
def count_parameters(model):
    """Count total and trainable parameters"""
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

total, trainable = count_parameters(model)

print("------ Model Information ------")
print(f"Total parameters: {total:,} ({total/1e9:.2f}B)")
print(f"Trainable parameters (before LoRA): {trainable:,}")
print(f"\nModel architecture:")
print(model)

------ Model Information ------
Total parameters: 4,540,600,320 (4.54B)
Trainable parameters (before LoRA): 1,050,939,392

Model architecture:
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_laye

In [15]:
print("Loading Tokenizer")
print("---------------------------------")

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("Tokenizer loaded successfully")
print(f"\nVocabulary Size: {len(tokenizer)}")
print(f"EOS token: {tokenizer.eos_token}, ID: {tokenizer.eos_token_id}")
print(f"PAD token: {tokenizer.pad_token}, ID: {tokenizer.pad_token_id}")

Loading Tokenizer
---------------------------------


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/8afb486c1db24fe5011ec46dfbe5b5dccdb575c2/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "dtype": "bfloat16",
  "eos_token_id": 128009,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": null,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_parameters": {
    "rope_theta": 500000.0,
    "rope_type": "default"
  },
  "tie_word_embeddings": false,
  "transformers_version": "5.0.0",
  "use_cache": true,
  "vocab_size": 128256
}



tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Tokenizer loaded successfully

Vocabulary Size: 128256
EOS token: <|eot_id|>, ID: 128009
PAD token: <|eot_id|>, ID: 128009


In [16]:
# Test the tokenizer
test_text = "Hello! How are you doing, Could you help me with some task?"
tokens = tokenizer(test_text, return_tensors="pt")

print(f"Original text: {test_text}")
print(f"\nTokenized IDs: {tokens['input_ids'][0].tolist()}")
print(f"Number of tokens: {len(tokens['input_ids'][0])}")
print(f"\nDecoded back: {tokenizer.decode(tokens['input_ids'][0])}")

Original text: Hello! How are you doing, Could you help me with some task?

Tokenized IDs: [128000, 9906, 0, 2650, 527, 499, 3815, 11, 16910, 499, 1520, 757, 449, 1063, 3465, 30]
Number of tokens: 16

Decoded back: <|begin_of_text|>Hello! How are you doing, Could you help me with some task?


In [17]:
lora_r = 64
lora_alpha = 16
lora_dropout = 0.05
target_modules = [
    'q_proj',
    'k_proj',
    'v_proj',
    'o_proj',
    'gate_proj',
    'up_proj',
    'down_proj'
    ]


peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules
)

# print lora config details
print(" LoRA configuration created:")
print(f"  - Rank (r): {lora_r}")
print(f"  - Alpha: {lora_alpha}")
print(f"  - Dropout: {lora_dropout}")
print(f"  - Target modules: {peft_config.target_modules}")
print(f"  - Task type: {peft_config.task_type}")

 LoRA configuration created:
  - Rank (r): 64
  - Alpha: 16
  - Dropout: 0.05
  - Target modules: {'q_proj', 'gate_proj', 'v_proj', 'o_proj', 'down_proj', 'k_proj', 'up_proj'}
  - Task type: CAUSAL_LM


In [18]:
# Training Params
output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = True
per_device_train_batch_size = 1
gradient_accumulation_steps = 4
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_seq_length = 2048
warmup_ratio = 0.03
group_by_length = True
save_steps = 50
save_total_limit=2
logging_steps = 10
max_steps = 200

In [19]:
sft_config = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    save_total_limit=save_total_limit,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    gradient_checkpointing=gradient_checkpointing,
    gradient_checkpointing_kwargs={'use_reentrant': False},

    dataset_text_field="text",
    max_length=max_seq_length,
    packing=False,
)

# Print all attributes as a dictionary
import pprint
pprint.pprint(sft_config.to_dict())

PyTorch: setting up devices
warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


{'accelerator_config': {'dispatch_batches': None,
                        'even_batches': True,
                        'gradient_accumulation_kwargs': None,
                        'non_blocking': False,
                        'split_batches': False,
                        'use_seedable_sampler': True},
 'activation_offloading': False,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'assistant_only_loss': False,
 'auto_find_batch_size': False,
 'average_tokens_across_devices': True,
 'batch_eval_metrics': False,
 'bf16': True,
 'bf16_full_eval': False,
 'chat_template_path': None,
 'completion_only_loss': None,
 'data_seed': None,
 'dataloader_drop_last': False,
 'dataloader_num_workers': 0,
 'dataloader_persistent_workers': False,
 'dataloader_pin_memory': True,
 'dataloader_prefetch_factor': None,
 'dataset_kwargs': None,
 'dataset_num_proc': None,
 'dataset_text_field': 'text',
 'ddp_backend': None,
 'ddp_broadcast_buffers': None,
 'ddp_bucket_cap_mb': None,
 '

In [20]:
print("Initializing SFTTrainer...")

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    processing_class=tokenizer,
    args=sft_config,
)

print("\n Trainer initialized successfully!")
print(f"  - Max sequence length: {max_seq_length}")
print(f"  - Training samples: {len(dataset):,}")
print(f"  - Training steps: {max_steps}")

Initializing SFTTrainer...


Adding EOS to train dataset:   0%|          | 0/9846 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/9846 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/9846 [00:00<?, ? examples/s]


 Trainer initialized successfully!
  - Max sequence length: 2048
  - Training samples: 9,846
  - Training steps: 200


In [21]:
# Check trainable parameters after LoRA injection
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params:,} || "
        f"all params: {all_param:,} || "
        f"trainable%: {100 * trainable_params / all_param:.4f}"
    )

print("=== LoRA Parameters ===")
print_trainable_parameters(trainer.model)

=== LoRA Parameters ===
trainable params: 167,772,160 || all params: 4,708,372,480 || trainable%: 3.5633


In [22]:
# Convert normalization layers to float32 for training stability
# This is important when using mixed precision with quantized models
for name, module in trainer.model.named_modules():
    if "norm" in name or "lm_head" in name:
        module = module.to(torch.float32)

print(" Normalization layers converted to float32 for stability")

 Normalization layers converted to float32 for stability


In [23]:
# Start training!
print("=" * 70)
print("STARTING TRAINING")
print("=" * 70)
print(f"\nTraining for {max_steps} steps...")
print(f"Saving checkpoints every {save_steps} steps to: {output_dir}")
print("\nYou can monitor progress via the logging output below.")
print("=" * 70)
print()

# Train the model
trainer.train()

print("\n" + "=" * 70)
print("TRAINING COMPLETE!")
print("=" * 70)

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 128009}.


STARTING TRAINING

Training for 200 steps...
Saving checkpoints every 50 steps to: ./results

You can monitor progress via the logging output below.



The following columns in the Training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 9,846
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 4
  Total optimization steps = 2,462
  Number of trainable parameters = 167,772,160


Step,Training Loss
10,1.452583
20,1.397838
30,1.496645
40,1.610237
50,1.742431
60,1.266032
70,1.191950
80,1.392547
90,1.502595
100,1.698606


Saving model checkpoint to ./results/checkpoint-50
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/8afb486c1db24fe5011ec46dfbe5b5dccdb575c2/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "dtype": "bfloat16",
  "eos_token_id": 128009,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": null,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_parameters": {
    "rope_theta": 500000.0,
    "rope_type": "default"
  },
  "tie_word_embeddings": false,
  "transformers_version": "5.0.0",
  "use_cache": true,
  "vocab_size": 1282


TRAINING COMPLETE!


In [24]:
# Save the final trained model
final_model_path = "./metta-llama3-8b-instruct-guanaco-lora-final"

print(f"Saving final model to: {final_model_path}")
trainer.save_model(final_model_path)

print("\n Model saved successfully!")
print(f"\nModel location: {final_model_path}")
print("\nYou can load this model later for inference.")

Saving model checkpoint to ./metta-llama3-8b-instruct-guanaco-lora-final


Saving final model to: ./metta-llama3-8b-instruct-guanaco-lora-final


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/8afb486c1db24fe5011ec46dfbe5b5dccdb575c2/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "dtype": "bfloat16",
  "eos_token_id": 128009,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": null,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_parameters": {
    "rope_theta": 500000.0,
    "rope_type": "default"
  },
  "tie_word_embeddings": false,
  "transformers_version": "5.0.0",
  "use_cache": true,
  "vocab_size": 128256
}

chat template saved in ./metta-llama3-8b-inst


 Model saved successfully!

Model location: ./metta-llama3-8b-instruct-guanaco-lora-final

You can load this model later for inference.
